In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
By_sentence_pos=pd.read_csv('By_sentence_pos.csv')

In [ ]:
classif = pd.read_excel('./feature_8topic.xlsx')
classif.columns=['service','place','clean','cp','comfort','sleep','environment','food']
classif=classif.to_dict()
service_list=np.array(list(classif['service'].values()))
classif['service']=service_list[service_list!='nan']
place_list=np.array(list(classif['place'].values()))
classif['place']=place_list[place_list!='nan']
clean_list=np.array(list(classif['clean'].values()))
classif['clean']=clean_list[clean_list!='nan']
cp_list=np.array(list(classif['cp'].values()))
classif['cp']=cp_list[cp_list!='nan']
comfort_list=np.array(list(classif['comfort'].values()))
classif['comfort']=comfort_list[comfort_list!='nan']
sleep_list=np.array(list(classif['sleep'].values()))
classif['sleep']=sleep_list[sleep_list!='nan']
food_list=np.array(list(classif['food'].values()))
classif['food']=food_list[food_list!='nan']
environment_list=np.array(list(classif['environment'].values()))
classif['environment']=environment_list[environment_list!='nan']
def word_in_Category(x,cat):
    cat_list=[]
    for word,pos in x:
        word=word.lower().strip()
        if word in classif[cat]:
            cat_list.append(word)
    if cat_list==[]:
        cat_list=None
    return(cat_list)
for cat in classif.keys():
    By_sentence_pos[f'word_in_{cat}']=By_sentence_pos['pos'].apply(lambda x:word_in_Category(x,cat))

In [ ]:
from collections import defaultdict
def find_category(classif,x):
    cate_list=[]
    for cate in classif.keys():
        same=set(x.keys()).intersection(set(classif[cate]))
        if len(same)>0:
            cate_list.append(cate)
    if len(cate_list)==0:
        cate_list= 'None'
    return(cate_list)

In [ ]:
By_sentence_pos['token_ckip']=By_sentence_pos['pos'].apply(lambda x:[i[0].upper().strip() for i in x ])
By_sentence_pos['Counter']=By_sentence_pos['token_ckip'].apply(Counter)
By_sentence_pos['category']=By_sentence_pos['Counter'].apply(lambda x: find_category(classif,x))

In [ ]:
opinion_word=pd.read_excel('./emotion_score_Frequecy_filtered.xlsx')
word_dict={}
for w,s in zip(opinion_word['word'],opinion_word['emotion_score']):
    word_dict[w]=s

In [ ]:
negation=pd.read_csv('./outsourced_data/negation.csv')
negation_list=list(negation.word)
##程度詞,可以利用DfX詞性的詞進行區分 
degree=pd.read_excel('./degree_classif_filtered.xlsx')
degree_dict={}
for strong in degree['strong']:
    degree_dict[strong]=4
for medium in degree['medium']:
    degree_dict[medium]=3
for weak in degree['weak']:
    degree_dict[weak]=2

In [ ]:
pos_list=['VB','VC','VF','VH','VHC','VI','VL','VJ','VK','A']##情緒分數需要留的詞性
def EMO_Degree(x,pos_list):
    output_list=[]
    for word,pos in x:
        if word in negation_list:
            output_list.append((word,pos,'negation'))
        elif pos in pos_list :
            if word_dict.get(word)!=None:
                output_list.append((word,pos,'emo_score',word_dict.get(word)))
            else:
                output_list.append((word,pos,'other'))
        elif word in degree_dict.keys():
            output_list.append((word,pos,'degree',degree_dict.get(word)))
        else:
            output_list.append((word,pos,'other'))
    return(output_list)
By_sentence_pos['emo_pos']=By_sentence_pos['pos'].apply(lambda x:EMO_Degree(x,pos_list))

In [ ]:
def emotion_score(df):
    total_score=0
    nega_Flag=1
    degree_Flag=1
    count=0
    for emo_pos in df.emo_pos:
        if emo_pos[2]=='emo_score':
            total_score=total_score+(emo_pos[3])*nega_Flag*degree_Flag
        if emo_pos[2]=='degree':
            degree_Flag=emo_pos[3]
        if emo_pos[2]=='negation':
            nega_Flag=-1
        if (emo_pos[2]=='other')&(emo_pos[1]!='DE'):##other or 的
            count+=1
            if count>2: ##間隔不超過兩個字
                nega_Flag=1
                degree_Flag=1
                count=0
    return(total_score)
By_sentence_pos['emotion_score']=By_sentence_pos.apply(emotion_score,axis=1)

In [ ]:
def checkPos(x,cate):
    output=[]
    if cate=='emo_score':
        for word in x:
            if word[2]==cate:
                output.append((word[0],word[1],word[3]))

    elif cate=='degree':
        for word in x:
            if word[2]==cate:
                output.append((word[0],word[1],word[3]))
    elif cate=='negation':
        for word in x:
            if word[2]==cate:
                output.append(word[0])
    if output==[]:
        output=None
    return(output)    
By_sentence_pos['has_emo_Score']=By_sentence_pos['emo_pos'].apply(lambda x: checkPos(x,"emo_score"))
By_sentence_pos['has_degree']=By_sentence_pos['emo_pos'].apply(lambda x: checkPos(x,"degree"))
By_sentence_pos['has_negation']=By_sentence_pos['emo_pos'].apply(lambda x: checkPos(x,"negation"))

In [ ]:
def count_Score(x,flag):
    count=0
    if x!=None:
        if flag>0:
            for w,p,s in x:
                if s>0:
                    count+=1
        else:
            for w,p,s in x:
                if s<0:
                    count+=1
    return(count)
By_sentence_pos['positive_count']=By_sentence_pos['has_emo_Score'].apply(lambda x: count_Score(x,1))
By_sentence_pos['negative_count']=By_sentence_pos['has_emo_Score'].apply(lambda x: count_Score(x,-1))

In [ ]:
def count_degree(x,flag):
    count=0
    if x!=None:
        for w,p,s in x:
            if s==flag:
                count+=1
    return(count)
By_sentence_pos['weak_count']=By_sentence_pos['has_degree'].apply(lambda x:count_degree(x,2))
By_sentence_pos['medium_count']=By_sentence_pos['has_degree'].apply(lambda x:count_degree(x,3))
By_sentence_pos['strong_count']=By_sentence_pos['has_degree'].apply(lambda x:count_degree(x,4))

In [ ]:
By_sentence_pos.to_csv('result/BY_sentence_emo_score_detail.csv',index=False)